# Hello Bluesky: Reading detectors and scanning

## Before we start, we need some simluated hardware to control

Start a new Terminal and copy/paste this line:

```
./demo_ioc.py
```

## Configuration

This code would normally go in a script automatically run at startup. The user would not have to worry about this.

In [ ]:
# Boilerplate setup...

%matplotlib notebook
from nslsii import configure_base

# Magically add a bunch of variables in our namespace.
configure_base(get_ipython().user_ns, 'temp', epics_context=False);

In [ ]:
# Represent the hardware in ophyd:

from ophyd import EpicsSignal, Device, Component

class Detector(Device):
    intensity = Component(EpicsSignal, 'demo:det', kind='hinted')
    exposure = Component(EpicsSignal, 'demo:exp', kind='config')
    
det = Detector(name='det')
motor = EpicsSignal('demo:mtr', name='motor')  # a very simple motor (not a full EpicsMotor)

Check that we can communicate with the hardware. (We should get ``True``.)

In [ ]:
det.connected

In [ ]:
motor.connected

In [ ]:
from bluesky.plans import count, scan

Put the cursor after ``import`` and hit the TAB key to see a list of plans. Or, better yet, see [this page of the bluesky documentation](https://nsls-ii.github.io/bluesky/plans.html).

In [ ]:
from bluesky.plans import 

## Data Acquisition

### Executing a `count` plan with various parameters

In [ ]:
RE(count([det]))

The return value is a list of the run IDs that uniquely identify this data set. The "scan num" is easier to remember but is not good for long-term reference because it may not be unique.

Let's looks at the documentation for `count` to see what our other options are.

In [ ]:
count?

In [ ]:
# five consecutive readings
RE(count([det], num=5))

In [ ]:
# five sequential readings separated by a 1-second delay
RE(count([det], num=5, delay=1))

In [ ]:
# multiple detectors
from ophyd.sim import det1, det2

RE(count([det1, det2], num=3))

### Scan

Scan ``motor`` from -10 to 10, stopping at 15 equally-spaced points along the way and reading ``det``.

In [ ]:
RE(scan([det], motor, -10, 10, 15))

### Custom plan

In [ ]:
# The plan_stubs module contains smaller plans.
# They can be used alone or as buildling blocks for larger plans.
from bluesky.plan_stubs import mv


def sweep_exposure_time(times):
    "Three runs with different exposure times."
    for t in times:
        yield from mv(det.exposure, t)
        yield from scan([det], motor, -10, 10, 15)
        
RE(sweep_exposure_time([0.1, 1, 10]))

## Exercises

Q1: Above we ran a `count` with multiple readings separated by a fixed delay. The ``delay`` parameter also accepts a list of values. Try a `count` with a variable delay.


In [ ]:
# Try your solution here. Fill in the blank:
# RE(count(____)))

Execute the following cell to reveal a solution:

In [ ]:
%load solutions/count_variable_delay.py

Q2: Write a custom plan that scans the same region twice, first with coarse steps and then with fine steps.

In [ ]:
# Try your solution here. Fill in the blank:
# def coarse_and_fine(detectors, motor, start, stop):
#     yield from scan(___)
#     yield from scan(___)
#
# RE(coarse_and_fine([det], motor, -10, 10))

In [ ]:
%load solutions/scan_coarse_and_fine.py

Q3: What's wrong with this? (What does it do?)
    
```python
def sweep_exposure_time(times):
    "Three runs with different exposure times."
    for t in times:
        mv(det.exposure, t)
        scan([det], motor, -10, 10, 15)
```

<details>
    <summary>Show answer</summary>
    The code runs, but no hardware is moved and no measurements are taken. The code is missing ``yield from`` from ``mv`` and ``scan``. Read "yield from" as meaning "consume instructions from". Don't forget to type it!
</details>